# Ejercicio 1

Por iteración, vamos a hacer lo siguiente:

1. Se generará una población de vectores 

1. **Primera iteración:** Se genera una población de $2k$ individuos aleatoria (cada individuo es un vector de números reales).

2. Se evalúan los individuos mediante la **función de aptitud** y se les asigna un 'pedazo del pastel' en la ruleta (proporcional a 'que tan buenos' son): 
    * Tendremos una `sumaAptitudes` **($F$)** que nos va a determinar 'el tamaño del pastel completo'. Éste es la suma de todas las aptitudes ($eval(v_i)$).
    * Para cada individuo $v_i$, tendremos una **probabilidad de selección asociada** $p_i$. Ésta está dada por: $\frac{eval(v_i)}{F}$.
    * El 'pedazo de pastel' que le corresponde a un individuo es su probabilidad de ser seleccionado para la siguiente generación. Ésta depende de su aptitud entre la suma de aptitudes. Vamos a considerar una probabilidad acumulativa para facilitar la implementación.
        * Por 'probabilidad acumulativa' nos referimos a:
        
            Si tienes una lista de individuos con sus probabilidades $p_i$ las probabilidades acumulativas $q_i$ crean una escala o 'intervalos' en los que se ubica cada individuo.

            Por ejemplo si el **Individuo 1** tiene una probabilidad de 0.1, y el **Individuo 2** tiene una probabilidad de 0.2, entonces: 
            
            $q_1 = 0.1$ (el intervalo del **Individuo 1** es de 0 a 0.1) 
            
            $q_2 = 0.1 + 0.2 = 0.3$ (el intervalo del **Individuo 2** es de 0.1 a 0.3)

3. Con probabilidad uniforme (pensando en que estamos lanzando un dardo a la ruleta) generamos un número $r \in [ 0,1 ]$. Si $r < q_i$ (si el dardo cae en el pedazo de pastel correspondiente al individuo $v_i$), el individuo $v_i$ es elegido como el primer padre. 
    
4. Aquí viene la parte de generación de crías.     
    * Para evitar la rápida pérdida de diversidad, no vamos a permitir un RadagonXMarika (no se vale generar hijos con un sólo padre). Seguro esto se puede hacer usando algún `if`.
    * Vamos a generar $2k-1$ hijos (pues en cada generación, la idea es quedarnos con el mejor y matar a los demás). Recordemos que cada pareja de padres nos da dos hijos. Por ello, vamos a pedirle al programa que elija $k$ parejas de padres y descarte a un hijo aleatorio (no le pediremos que descarte 'al peor' porque eso propiciará la rápida pérdida de diversidad).
    * En este paso, vamos a hacer un paso intermedio donde se codifica a los padres como vectores binarios y por cromosoma vamos a aplicar el operador de reproducción correspondiente.
        * Nuestra implementación es algo del estilo: `[[0,1,1,0,...], ..., [0,1,1,1,...]]`. Por 'cromosoma' nos referimos a cada uno de los vectores de 0s y 1s (pues en nuestra implementación, los vectores de números reales pasan a ser vectores de vectores de 0s y 1s).

5. Antes de actualizar la nueva generación, vamos a pasar la lista de los $2k - 1$ hijos por el operador de mutación flip (en principio 1-flip). **ESTO VISÍTALO MÁS ADELANTE**.

6. Tenemos una nueva población de $2k$ individuos a la que le vamos a repetir el proceso.



Tenemos un total de 5 funciones de prueba y cada una tiene intervalos de búsqueda diferentes. 

In [5]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valorios aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   # Para facilitarnos la vida, convertimos a la matriz en una lista
   # de listas:
   poblacionLista = poblacionMatriz.tolist()

   return poblacionLista


In [6]:
'''Veamos un ejemplo de implementación'''

n = 3 # Dimensión en la que están los vectores
k = 2 # Queremos 4 individuos.
intervalo = (-5, 5) # Para efectos del ejemplo, usaremos el intervalo de
                    # la función Rastrigin.

poblacionInicial = Generar_Poblacion(k, n, intervalo)

print(f"Problación generada:\n {poblacionInicial}")
print(f"De {len(poblacionInicial)} individuos.")


Problación generada:
 [[-0.2640147525594285, -4.6718874782300714, -2.185544271447779], [4.515882325130637, 2.5300437267558404, -3.612951582330093], [2.2302938281772704, -1.8730305654241586, -3.1210313519103297], [0.8724555342800864, -0.9890780201330607, 2.092897134418142]]
De 4 individuos.


Necesitamos una **función de aptitud** para poder calcular el 'tamaño del pastel'.

Por *razones* (véase el PDF de 'Medida_proba.pdf') -a reserva de que computacionalmente hablando sea demasiado caro- vamos a usar $g(v_i)=e^{-f(v_i)}$ como nuestra función de aptitud.

De este modo, el 'tamaño del pastel' está definido por $F = \sum g(v_i) = \sum e^{-f(v_i)}$.

Para cada $v_i$, $p_i = \frac{e^{-f(v_i)}}{\sum e^{-f(v_i)}} $.

Y para cada $v_i$, su probabilidad acumulativa es $q_i = \sum_{j=i}^{i} \frac{e^{-f(v_j)}}{\sum e^{-f(v_j)}} $.


In [7]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y escupe una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''

def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion):
    # Generamos una lista con las evaluaciones en la función objetivo
    evaluaciones = [funcion(individuo) for individuo in poblacion]

    #print("Evaluaciones:", [float(eval) for eval in evaluaciones], "\n")

    # Generamos una lista de las aptitudes de los individuos en la 
    # población.
    aptitudes = [np.exp(-evaluacion) for evaluacion in evaluaciones]

    #print("Aptitudes:", [float(apt) for apt in aptitudes], "\n")

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    #print("El individuo más apto fue:", masApto, "\n")

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = [(aptitud/sumaAptitudes) for aptitud in aptitudes]

    #print("Probabilidades de selección:", [float(pi) for pi in probabilidades])


    return probabilidades, masApto


In [8]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 1 # Dimensión en la que están los vectores
k = 2 # Queremos 4 individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}")


Problación generada:
[[4.214951823559506], [-0.6138700557809464], [-0.43002853900808713], [-0.129551382294677]]

De 4 individuos.

Las evaluaciones fueron:
[25.581432446240587, 17.92470852193232, 19.233958413415408, 3.150791933482254]

El individuo más apto fue [-0.129551382294677]

Las probabilidades de selección fueron
[1.8134115943957745e-10, 3.8350350537839717e-07, 1.035545854955969e-07, 0.999999512760568]


**HASTA AQUí TENEMOS...**

1. Una manera de generar de manera aleatoria una población inicial.
2. La lista de las $p_i$ y el mejor individuo de la población (esto es iterable).

**EL SIGUIENTE PASO ES:**

Generar las probas acumulativas para poder hacer la implementación de la elección de los padres.

In [9]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos escupe la que corresponde a la de las 
   probabilidades acumulativas.'''

def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = []
    sumaAcumulada = 0

    for proba in probabilidades:
        sumaAcumulada += proba
        probasAcumulativas.append(sumaAcumulada)

    #return probasAcumulativas
    return probasAcumulativas


In [10]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 1 # Dimensión en la que están los vectores
k = 2 # Queremos 4 individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")


Problación generada:
[[3.9849320275475737], [-2.812617927617158], [4.869710952327614], [-3.09671942246586]]

De 4 individuos.

Las evaluaciones fueron:
[15.924466447177375, 14.077140746556605, 26.88186407355519, 11.38007131095584]

El individuo más apto fue [-3.09671942246586]

Las probabilidades de selección fueron
[0.00985742789882992, 0.06252403063576685, 1.718012360079936e-07, 0.9276183696641673]

Las probabilidades acumulativas fueron
[0.00985742789882992, 0.07238145853459677, 0.07238163033583277, 1.0]



Necesitamos ahora una función que haga la elección de los padres:

Usamos `tol=1e-9` y `math.isclose()` porque las comparaciones de números de punto flotante (números decimales) pueden ser imprecisas, debido a cómo las computadoras los manejan. 

`math.isclose()` permite manejar esta imprecisión al verificar si `r` y `probAcumulada` son prácticamente iguales. Esto evita errores en situaciones donde `r` cae "justo en el filo" que separa las probabilidades acumulativas entre dos individuos.

`tol=1e-9`  define la tolerancia. Si `r` está a una distancia menor que $10^{-9}$ de `probAcumulada`, lo tratamos como si fueran iguales.

En general, en este tipo de algoritmos, cuando $r$ cae exactamente en el límite entre dos intervalos, lo más común es que se elija el individuo cuyo intervalo incluye el valor en el extremo inferior. 

Esto significa que si $r$ es exactamente igual a la probabilidad acumulativa de un individuo, se selecciona el individuo más pequeño, es decir, el que corresponde a ese valor acumulativo. O sea, los intervalos son inclusivos en su límite inferior y exclusivos en su límite superior. Por ejemplo, un intervalo $[0.2, 0.5)$ incluiría 
$r=0.2$, pero no $r=0.5$.

In [11]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

import random
import math
import time

def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   for i, probAcumulada in enumerate(probasAcumuladas):
      if r<= probAcumulada or math.isclose(r, probAcumulada, abs_tol=tol):
         return i # Índice del individuo seleccionado.
      


'''Le segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k-1 nuevos individuos, así que se van a 
   elegir 2k-1 parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades aculativas y tiempo máximo de ejecución para intentar
   evitar la generación de clones (en este caso, vamos a considerar un máximo
   de 10 segundos para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''
   

def Generar_Parejas(individuos, probasAcumuladas, maxTiempo=5):
   
   parejas = [] # Aquí guardamos a la lista de parejas que nos darán a
                # la siguiente generación.

   # Como len(individuos)=2k, le pedimos que elija a len(individuos)/2
   # parejas de padres (que nos generarán 2k hijos).
   parejasNecesarias = int(len(individuos)/2)  

   clonesGenerados = 0 # Sólo por curiosidad, vamos a llevar un
                       # contador de clones.

   for _ in range(parejasNecesarias):
      
      padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

      tiempoInicio = time.time() # Registramos el tiempo de inicio.

      # En principio, no queremos clones. Se intentará elegir un segundo 
      # padre distinto y una vez que hayamos rebasado el máximo de tiempo
      # nos resignamos y elegimos un clon.
      while True:
         
         padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

         if padre1_idx != padre2_idx: # Comparamos los índices y si 
                                      # tenemos padres distintos, terminamos.
            break
      
      # Si el tiempo transcurrido supera maxTiempo, permitimos un clon
      if time.time() - tiempoInicio > maxTiempo: 
         padre2_idx = padre1_idx
         clonesGenerados +=1 # Agregamos el clon al contador.
         break
      
      # Guardamos la pareja de padres usando sus índices:
      parejas.append((individuos[padre1_idx], individuos[padre2_idx]))

   return parejas, clonesGenerados


**Sugerencia de ChatGPT:**

In [12]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

import random
import math

def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   for i, probAcumulada in enumerate(probasAcumuladas):
      if r<= probAcumulada or math.isclose(r, probAcumulada, abs_tol=tol):
         return i # Índice del individuo seleccionado.


def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    parejas = []
    clonesGenerados = 0  # Contador de clones generados
    numIndividuos = len(individuos)

    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return parejas, clonesGenerados

    # Como len(individuos)=2k, le pedimos que elija a len(individuos)/2
    # parejas de padres (que nos generarán 2k hijos).
    parejasNecesarias = int(numIndividuos/2)
    #print(f"Se necesitan {parejasNecesarias} parejas de padres.")

    for _ in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salir del ciclo
            
            intentos += 1
            if intentos >= maxIntentos:  # Después de varios intentos, permitir clones
                padre2_idx = padre1_idx
                clonesGenerados += 2 # Cada pareja produce dos hijos
                #print(f"Se ha generado un clon tras {intentos} intentos.")
                break  # Salir del ciclo permitiendo el clon

        # Agregamos la pareja solo si hay padres válidos
        if padre1_idx is not None and padre2_idx is not None:
            parejas.append((individuos[padre1_idx], individuos[padre2_idx]))
        else:
            print("Error: No se pudieron seleccionar padres válidos.")

    return parejas, clonesGenerados


In [13]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 2 # Dimensión en la que están los vectores
k = 2 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, numDeClones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generar {int(len(parejas)*2)} nuevos individuos.\n")
print(f"Se permitirá un total de {numDeClones} clones.\n")


Problación generada:
[[-4.828910468729294, 0.003021065683799229], [-3.05297614808973, -1.7191623118356656], [-0.2811128145816362, -1.4280320249508902], [2.9736029322363757, 1.804741795280548]]

De 4 individuos.

Las evaluaciones fueron:
[28.56275218965549, 24.750550043550284, 33.055680276938645, 18.864525770776172]

El individuo más apto fue [2.9736029322363757, 1.804741795280548]

Las probabilidades de selección fueron
[6.121841763666066e-05, 0.0027701319463965526, 6.849016680862628e-07, 0.9971679647342987]

Las probabilidades acumulativas fueron
[6.121841763666066e-05, 0.002831350364033213, 0.0028320352657012996, 1.0]

Los padres de la siguiente generación son:
[([2.9736029322363757, 1.804741795280548], [-3.05297614808973, -1.7191623118356656]), ([2.9736029322363757, 1.804741795280548], [2.9736029322363757, 1.804741795280548])]

Tenemos 2 parejas de padres, pues queremos generar 4 nuevos individuos.

Se permitirá un total de 2 clones.



**POR EL MOMENTO VAMOS A USAR LA SUGERENCIA DE CHATGPT**

**HASTA AQUí TENEMOS...**

1. Una manera de generar de manera aleatoria una población inicial.
2. La lista de las $p_i$ y el mejor individuo de la población (esto es iterable).
3. La lista de las $q_i$ (esto es iterable también).
4. Tenemos la elección de los padres por medio de la ruleta (esto también es iterable).

**EL SIGUIENTE PASO ES:**

Codificar las parejas de padres como vectores binarios e implementar los distintos operadores de reproducción usando bits.

In [14]:
'''Funciones auxiliares para la codificación en binario.'''

def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando nBit bits y una partición uniforme en [a, b].

    Parámetros:
    x: número a real a codificar.
    nBit: número de bits a utilizar.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con nBit bits en el intervalo [a, b].
    """
    
    a, b = intervalo
    
    # Calcula la precisión de la representación.
    precision = (b - a) / (2 ** n_bit)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario usando nuestro codigo para codificar naturales.
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = [0] * n_bit
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    # Devuelve el arreglo binario que representa al número real 'x'.
    return x_binario

def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios utilizando nBit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    dim_x: Diimensión del vector vector_reales
    nBit: Número de bits a utilizar para las entradas de nuestro arreglo.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo de arreglos arreglos binarios, donde cada subarreglo representa un número real codificado en binario
    """
    
    dim_x = len(vector_reales)

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = []

    # Itera sobre cada entrada de nuestro vector de reales.
    for i in range(dim_x):
        numero = vector_reales[i]

        # Codifica cada número utilizando la función codifica
        binario = codifica_real(numero, n_bit, intervalo)

        # Añade el resultado codificado al vector principal
        vector_binario.append(binario)

    # Devuelve un arreglo de arreglos binarios.
    return vector_binario


print(f"El vector {[1.4,1.2,3,1.5674]} en binario con {10} bits es:\n")
print(codifica_vector([1.4,1.2,3,1.5674],10,(1,5)))


El vector [1.4, 1.2, 3, 1.5674] en binario con 10 bits es:

[[0, 0, 0, 1, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 1]]


In [15]:
'''Primero, viene la codificación en vectores binarios del estilo
   [[0,1,1,1,...], ..., [1, 1, 1, 0,...]]'''

def Padres_Binarios(parejas, nBits, intervalo):
   parejasBinarias = []

   for padre1, padre2 in parejas:

      padre1Bin = codifica_vector(padre1, nBits, intervalo)
      padre2Bin = codifica_vector(padre2, nBits, intervalo)

      parejasBinarias.append((padre1Bin, padre2Bin))
   
   return parejasBinarias


In [16]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 2 # Dimensión en la que están los vectores
k = 3 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, clones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generar {int(len(parejas)*2)} nuevos individuos.\n")
print(f"Se permitirán {clones} clones\n")

nBits = 6 # Vamos a considerar nBits bits
parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres de la siguiente generación (como vectores binarios) son:\n{parejasBinarias}\n")


Problación generada:
[[4.580935327337901, -2.789935894256501], [0.508254148745694, 3.0425275978535335], [-3.238290270764985, 1.4466903595196783], [-4.742366095858114, 1.1179001131090462], [-4.688461842558308, 3.0451190152067813], [4.262466457065354, -3.3017476198931863]]

De 6 individuos.

Las evaluaciones fueron:
[55.0203172061337, 29.856734485504923, 41.28860966515673, 36.839837132316816, 45.424507094813336, 53.047060963614186]

El individuo más apto fue [0.508254148745694, 3.0425275978535335]

Las probabilidades de selección fueron
[1.1781145285226266e-11, 0.9990624408358498, 1.083408712877719e-05, 0.0009265517617389489, 1.7321874689423512e-07, 8.475431325854448e-11]

Las probabilidades acumulativas fueron
[1.1781145285226266e-11, 0.999062440847631, 0.9990732749347597, 0.9999998266964987, 0.9999999999152456, 0.9999999999999999]

Los padres de la siguiente generación son:
[([0.508254148745694, 3.0425275978535335], [0.508254148745694, 3.0425275978535335]), ([0.508254148745694, 3.04252

Vamos ahora a considerar los operadores de generación de crías, estos los vamos a aplicar cromosoma a cromosoma (O sea, los puntos de cruza se van a considerar dentro de cada uno de los vectores de 0s y 1s)

**Operador de cruza de $n$ puntos**

1. En nuestra codificación de los vectores, cada cromosoma tiene una cierta cantidad de `nBits` bits, la $n<=(nBits-1)$. Si tienes $6$ pedazos de un pastel, quiere decir que hiciste $5=6-1$ cortes.
2. Los puntos de corte deben ser únicos y generados de manera aleatoria. No queremos que se nos genere dos o más veces un mismo punto de corte.

In [17]:
def Cruzar_N_Puntos(padresBinarios, nCortes, nBits):
    """
    Realiza cruza de n puntos entre las parejas de padres.
    
    :param padresBinarios: Lista de parejas de padres. Cada padre es una lista de cromosomas (binarios).
    :param nCortes: Número de puntos de cruza.
    :param nBits: Bits de la codificación de los padres en binario.
    :return: Lista de hijos generados (2 hijos por pareja de padres).
    """
    hijos = []
    
    # Los `nCortes` son un parámetro recibido e idealmente es el que se
    # usará, en caso de que `nCortes` sea demasido grande, se hará el 
    # mayor número de cortes posible, o sea `nBits - 1`
    n = min(nCortes, (nBits - 1))
    
    for padre1, padre2 in padresBinarios:
        
        # Los hijos se van a crear cruzando cada cromosoma de los padres. 
        # Recordemos que cada pareja produce dos hijos.
        hijo1 = []
        hijo2 = []
        
        # Cruzamos cada cromosoma individualmente
        for cromo1, cromo2 in zip(padre1, padre2): # `zip(padre1, padre2)` toma los cromosomas 
                                                   # correspondientes de ambos padres al mismo 
                                                   # tiempo, es decir, empareja los cromosomas 
                                                   # que están en las mismas posiciones de las 
                                                   # listas `padre1` y `padre2`.
            
            # Determinar los puntos de cruce
            puntosCruce = sorted(random.sample(range(1, len(cromo1)), n))

            #print(puntosCruce) # En la primera iteración, si tenemos [1, 5]
                               # esto quiere decir que en el primer cromosoma
                               # de la pareja de padres, se van a hacer cruces
                               # en tres pedazos determinados por los dos cortes
                               # en las posiciones 1 y 5.
            
            # Inicializamos cromosomas vacíos para los hijos
            hijo1Cromo = []
            hijo2Cromo = []
            
            ultimoPunto = 0 # Punto en el que vamos para los cruces, inicia en 0.
            
            switch = False  # Esta bandera indica si debemos cambiar de padre
            
            # Alternamos entre segmentos de los padres en los puntos de cruza
            for punto in (puntosCruce + [len(cromo1)]): # Añadimos el valor len(cromo1) 
                                                        # al final de la lista de puntos de 
                                                        # cruza para asegurarnos de que el 
                                                        # último segmento después del último 
                                                        # punto de corte también se incluya. 
                                                        # De esta manera, recorremos todo el 
                                                        # cromosoma.
                
                # Seleccionamos los segmentos entre el último punto y el punto actual
                if switch:
                    hijo1Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                    hijo2Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                else:
                    hijo1Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                    hijo2Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                
                # Cambiamos el segmento
                switch = not switch # De esta manera garantizamos que vamos a 
                                    # ir alternando.
                ultimoPunto = punto # Nos movemos al siguiente punto de corte.
            
            # Añadimos el nuevo cromosoma a los hijos
            hijo1.append(hijo1Cromo)
            hijo2.append(hijo2Cromo)
        
        # Añadimos los dos hijos generados a la lista de hijos
        hijos.append(hijo1)
        hijos.append(hijo2)
    
    # Para este punto ya tenemos la lista de los hijos, así que lo que toca es
    # eliminar uno para hacerle espacio al individuo más apto.
    index_a_eliminar = random.randint(0, len(hijos)-1) # De manera aleatoria, 
                                                       # elegimos un índice para
                                                       # quitar un hijo de la lista.
    #print(hijos)
    #print(f'Eliminando al hijo en la posición: {index_a_eliminar}')
    del hijos[index_a_eliminar] # Borramos al hijo elegido.
    
    return hijos # Retornamos una lista de 2k-1 hijos (que era lo que queríamos)

# Ejemplo de uso
# Padres: lista de k parejas de padres (cada padre tiene varios cromosomas binarios)
padres = [
    ([[1, 0, 1, 1, 0, 1], [1, 1, 0, 0, 1, 1]], [[0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 1, 0]]),
    ([[1, 1, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1]], [[0, 1, 1, 1, 0, 0], [1, 1, 0, 0, 1, 1]])
]

nCortes = 2  # Por ejemplo, cruza con 2 puntos

# Generamos a los hijos
hijos = Cruzar_N_Puntos(padres, nCortes, nBits=6)

# Mostrar los hijos
for i, hijo in enumerate(hijos, 1):
    print(f'Hijo {i}: {hijo}')

print(hijos)


Hijo 1: [[1, 0, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1]]
Hijo 2: [[1, 1, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1]]
Hijo 3: [[0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 1]]
[[[1, 0, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1]], [[1, 1, 0, 1, 1, 1], [0, 1, 0, 0, 1, 1]], [[0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 1]]]


In [18]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 2 # Dimensión en la que están los vectores
k = 2 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, clones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generar {int(len(parejas)*2)} nuevos individuos.\n")
print(f"Se permitirán {clones} clones\n")

nBits = 3 # Vamos a considerar nBits bits
parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres de la siguiente generación (como vectores binarios) son:\n{parejasBinarias}\n")

nCortes = 1 # Vamos a hacer cruza de un punto
hijosBinarios = Cruzar_N_Puntos(parejasBinarias, nCortes, nBits) # Generamos a los hijos
print(f"Los hijos generados en la cruza (como vectores binarios) son:\n{hijosBinarios}\n")
print(f"Se generaron: {len(hijosBinarios)} hijos")


Problación generada:
[[0.2656548213163221, 4.233134196001199], [0.86397338406152, 1.9589884483475544], [-1.2246785797167723, -4.624636525289776], [4.356413089510852, 4.96139098419799]]

De 4 individuos.

Las evaluaciones fueron:
[37.914306261731056, 8.349659703089083, 48.39001048884644, 60.085522616704644]

El individuo más apto fue [0.86397338406152, 1.9589884483475544]

Las probabilidades de selección fueron
[1.4462292475529148e-13, 0.9999999999998553, 4.080342325214824e-18, 3.399383954143184e-23]

Las probabilidades acumulativas fueron
[1.4462292475529148e-13, 1.0, 1.0, 1.0]

Los padres de la siguiente generación son:
[([0.86397338406152, 1.9589884483475544], [0.86397338406152, 1.9589884483475544]), ([0.86397338406152, 1.9589884483475544], [0.86397338406152, 1.9589884483475544])]

Tenemos 2 parejas de padres, pues queremos generar 4 nuevos individuos.

Se permitirán 4 clones

Los padres de la siguiente generación (como vectores binarios) son:
[([[1, 0, 0], [1, 0, 1]], [[1, 0, 0], [1

Ahora, en la lista de $2k-1$ hijos, vamos a implementar un operador de mutación 1-flip.
Lo haremos de la siguiente forma: 
1. Para cada hijo elegimos un cromosoma al azar.
2. Dentro del cromosoma elegido, elegimos un bit al azar.
3. Tenemos una cierta probabilidad de mutación $p_m$, generamos un número aleatorio $r$ entre 0 y 1. Si $r<p_m$, entonces el bit elegido cambia su valor ($0 \to 1$ y $1 \to 0$).
4. Generamos una nueva lista de hijos (posiblemente) mutados.

Aunque nuestras implementaciones son en dimensiones 'grandes' (dimensión 10), nos gustaría que el programa también funcione aún cuando tengamos listas de hijos donde cada hijo es directamente un cromosoma (en el caso en el que estemos trabajando en $\mathbb{R}$)

In [19]:
'''Recibimos como parámetros: la lista de los hijos (en binario) y una probabilidad
   de mutación. La función retorna la lista (en binario) de los hijos que (posiblemente)
   han sido mutados.'''

import copy

def Mutador_1_flip(hijos, probaMutar):
    
    hijosMutados = []
    #huboMutacion = False # Variable para verificar si hubo al menos una mutación.

    for hijo in hijos:
        hijo = copy.deepcopy(hijo)  # Hacemos una copia profunda del hijo para no modificar el 
                                    # original. si no hacemos una copia de los hijos antes de 
                                    # realizar mutaciones, estaremos modificando las listas 
                                    # originales directamente. Lo que puede causarnos problemas
                                    # si luego intentamos usar los hijos originales en alguna 
                                    # otra operación.          

        if isinstance(hijo[0], list):  # Si el primer elemento es una lista, tenemos listas de 
                                       # cromosomas (o sea, la dimensión del problema es al 
                                       # menos 2).
            # Elegimos un cromosoma al azar dentro del hijo

            cromElegido_index = random.randint(0, len(hijo)-1) # Elegimos un índice al azar en 
                                                           # `hijo`. Este corresponde a un 
                                                           # cromosoma.
            cromElegido = hijo[cromElegido_index] # Fijamos el cromosoma elegido.

            bit_index = random.randint(0, len(cromElegido)-1) # Elegimos un índice al azar en 
                                                              # el cromosoma elegido, este 
                                                              # corresponde a alguno de los 
                                                              # nBits.
        
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
            
                # Hacemos flip al bit: si es 0 lo cambiamos a 1, si es 1 lo cambiamos a 0
                cromElegido[bit_index] = 1 if cromElegido[bit_index] == 0 else 0

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)

        else: # Si el  no es una lista de listas, es directamente un cromosoma (o sea, la
              # dimensión del problema es 1).
        
            bit_index = random.randint(0, len(hijo)-1) # Elegimos un bit al azar dentro del 
                                                       # cromosoma (en este caso, cada hijo
                                                       # es su propio cromosoma).
            
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
                hijo[bit_index] = 1 if hijo[bit_index] == 0 else 0 # Flip del bit.

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)
    
    #print("Hubo mutación:", huboMutacion)

    return hijosMutados

# Ejemplo de uso:
hijos = [[1, 0, 1, 0, 1, 1], [1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 0]]
p_m = 0.3  # Probabilidad de mutación del 20%

nuevos_hijos = Mutador_1_flip(hijos, p_m)
print("Hijos originales:", hijos)
print("Hijos después de la mutación:", nuevos_hijos)


Hijos originales: [[1, 0, 1, 0, 1, 1], [1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 0]]
Hijos después de la mutación: [[1, 0, 1, 1, 1, 1], [1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 0]]


In [20]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 10 # Dimensión en la que están los vectores
k = 120 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, clones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generar {int(len(parejas)*2)} nuevos individuos.\n")
print(f"Se permitirán {clones} clones\n")

nBits = 15 # Vamos a considerar nBits bits
parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres de la siguiente generación (como vectores binarios) son:\n{parejasBinarias}\n")

nCortes = 20 # Vamos a hacer cruza de un punto
hijosBinarios = Cruzar_N_Puntos(parejasBinarias, nCortes, nBits) # Generamos a los hijos
print(f"Los hijos generados en la cruza (como vectores binarios) son:\n{hijosBinarios}\n")
print(f"Se generaron: {len(hijosBinarios)} hijos.\n")

probaMutar = 0.3 # Probabilidad de mutación.
hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)
print(f"Los hijos (posiblemente) mutados son:\n{hijosMutados}\n")
print(f"Tenemos {len(hijosMutados)} hijos.\n")



Problación generada:
[[-4.984948966607346, 4.298485390073848, 3.852923202703117, 3.862953548587301, -0.5870258963357671, -4.777801633975721, 4.878777284785727, -3.109079188452597, 3.1752493675011486, -0.15455643643237238], [-4.118510871046133, 1.8028250514055255, 3.3231075191062924, -1.534818590490298, -1.2299727872427213, -0.5205767894519227, 4.015419273227209, -0.5344250401656634, -4.5767774163995325, -3.077981301416284], [0.654414618779132, -0.0054336830135301994, 3.3743030805862357, 3.5856973363623705, 0.7371821103701857, 2.297001436410884, -4.942510318228749, -1.4363670278960239, -4.854206585811553, -4.523829210891068], [-0.7352807458735819, 4.104438151374305, 2.13742632638854, 0.5258120865994433, -0.35715265285498354, 4.673580450505121, -2.5098244772596234, 2.518001377964657, 1.2731362044609043, -3.8608815836869894], [1.2647429822946785, 3.3719874550432465, -3.5927942441810847, -3.865066794509816, -4.1295104083609315, -3.2583469757335592, 1.3597151714279185, -2.7379505065471967, 

Vamos ahora a pasar a los hijos (posiblemente) mutados de su codificación en arreglos de bits a vectores de números reales.

In [21]:
'''Nuestras funciones auxiliares:'''

def decodifica(x_cod, nBits, intervalo):
    """
    Decodifica un vector de bits como un número real en el intervalo [a, b].

    Parámetros:
    x_cod: vector de bits a decodificar
    nBits: número de bits a utilizar
    intervalo: De la forma [a, b]

    Retorno:
    número real decodificado
    """

    a, b = intervalo
    precision = (b - a) / (2 ** nBits)  # Calcula la precisión de la representación
    indice = int(''.join(map(str, x_cod)), 2)  # Convierte el vector de bits en un número entero
    x_dec = a + indice * precision  # Calcula el número real decodificado

    return x_dec

def decodifica_vector(vector_binario, nBits, intervalo):
    """
    Decodifica un vector de arreglos binarios en un vector de números reales.

    Parámetros:
    vector_binario: lista de listas de enteros, donde cada sublista representa un número codificado en binario
    nBits: número de bits a utilizar para cada número
    a: límite inferior de la partición
    b: límite superior de la partición

    Retorno:

    lista de números reales decodificados
    """

    dim_x = len(vector_binario)


    vector_reales = []

    for i in range(dim_x):
        binario = vector_binario[i]
        # Decodifica cada número utilizando la función decodifica
        real = decodifica(binario, nBits, intervalo)
        # Añade el resultado decodificado al vector principal
        vector_reales.append(real)

    return vector_reales


In [22]:
'''Función que decodifica a los hijos mutados para el reemplazo en la siguiente generación.
   Recibe como parámetros a loss hijos (posiblemente) mutados (codificados en bits), la 
   cantidad de nBits usada en la codificación y el intervalo de nuestra función objetivo.
   Retorna la lista de los hijos como vectores de números reales.'''

def Hijos_Decodificados(hijosMutados, nBits, intervalo):
    hijosDecodificados = []
    for hijoMutado in hijosMutados:
        hijoDecodificado = decodifica_vector(hijoMutado, nBits, intervalo)
        hijosDecodificados.append(hijoDecodificado)

    return(hijosDecodificados)


In [23]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 1 # Dimensión en la que están los vectores
k = 2 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, clones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generar {int(len(parejas)*2)} nuevos individuos.\n")
print(f"Se permitirán {clones} clones\n")

nBits = 3 # Vamos a considerar nBits bits
parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres de la siguiente generación (como vectores binarios) son:\n{parejasBinarias}\n")

nCortes = 20 # Vamos a hacer cruza de un punto
hijosBinarios = Cruzar_N_Puntos(parejasBinarias, nCortes, nBits) # Generamos a los hijos
print(f"Los hijos generados en la cruza (como vectores binarios) son:\n{hijosBinarios}\n")
print(f"Se generaron: {len(hijosBinarios)} hijos.\n")

probaMutar = 0.3 # Probabilidad de mutación.
hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)
print(f"Los hijos (posiblemente) mutados son:\n{hijosMutados}\n")
print(f"Tenemos {len(hijosMutados)} hijos.\n")

hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
print(f"Los hijos decodificados son:\n{hijosDecodificados}\n")


Problación generada:
[[3.909180993028258], [-2.030420245994713], [4.919467494678923], [-1.5609719747612982]]

De 4 individuos.

Las evaluaciones fueron:
[16.866101952571626, 4.304715878191133, 25.454261307124625, 21.71174339196616]

El individuo más apto fue [-2.030420245994713]

Las probabilidades de selección fueron
[3.5047561660716894e-06, 0.9999964670345031, 6.529314547575573e-10, 2.755639935736126e-08]

Las probabilidades acumulativas fueron
[3.5047561660716894e-06, 0.9999999717906691, 0.9999999724436006, 1.0]

Los padres de la siguiente generación son:
[([-2.030420245994713], [-2.030420245994713]), ([-2.030420245994713], [-2.030420245994713])]

Tenemos 2 parejas de padres, pues queremos generar 4 nuevos individuos.

Se permitirán 4 clones

Los padres de la siguiente generación (como vectores binarios) son:
[([[0, 1, 0]], [[0, 1, 0]]), ([[0, 1, 0]], [[0, 1, 0]])]

Los hijos generados en la cruza (como vectores binarios) son:
[[[0, 1, 0]], [[0, 1, 0]], [[0, 1, 0]]]

Se generaron: 3

In [24]:
'''Función que nos da la siguiente generacion completa:'''

def Siguiente_Gen(hijosDecodificados, masApto):
    siguenteGeneracion = hijosDecodificados + [masApto]
    return siguenteGeneracion


In [26]:
'''Veamos un ejemplo de implementación'''

# Consideremos a la función Rastrigin como función de prueba.
def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma


funcion = Rastrigin
n = 1 # Dimensión en la que están los vectores
k = 2 # Queremos 2k individuos.
intervalo = (-5, 5) # Usaremos el intervalo de la función Rastrigin.
poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"Problación generada:\n{poblacionInicial}\n")
print(f"De {len(poblacionInicial)} individuos.\n")


evaluaciones = [funcion(individuo) for individuo in poblacionInicial]
evaluacionesReales = [float(eval) for eval in evaluaciones]

print(f"Las evaluaciones fueron:\n{evaluacionesReales}\n")

probabilidades, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)

probabilidadesReales = [float(pi) for pi in probabilidades]

print(f"El individuo más apto fue {masApto}\n")
print(f"Las probabilidades de selección fueron\n{probabilidadesReales}\n")

probasAcumuladas = Probas_Acumulativas(probabilidades)
probasAcumuladasReales = [float(qi) for qi in probasAcumuladas]

print(f"Las probabilidades acumulativas fueron\n{probasAcumuladasReales}\n")

parejas, clones = Generar_Parejas(poblacionInicial, probasAcumuladas)

print(f"Los padres de la siguiente generación son:\n{parejas}\n")
print(f"Tenemos {len(parejas)} parejas de padres, pues queremos generar {int(len(parejas)*2)} nuevos individuos.\n")
print(f"Se permitirán {clones} clones\n")

nBits = 3 # Vamos a considerar nBits bits
parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres de la siguiente generación (como vectores binarios) son:\n{parejasBinarias}\n")

nCortes = 20 # Vamos a hacer cruza de un punto
hijosBinarios = Cruzar_N_Puntos(parejasBinarias, nCortes, nBits) # Generamos a los hijos
print(f"Los hijos generados en la cruza (como vectores binarios) son:\n{hijosBinarios}\n")
print(f"Se generaron: {len(hijosBinarios)} hijos.\n")

probaMutar = 0.3 # Probabilidad de mutación.
hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)
print(f"Los hijos (posiblemente) mutados son:\n{hijosMutados}\n")
print(f"Tenemos {len(hijosMutados)} hijos.\n")

hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
print(f"Los hijos decodificados son:\n{hijosDecodificados}\n")

siguienteGeneracion = Siguiente_Gen(hijosDecodificados, masApto)
print(f"La siguiente generación es:\n{siguienteGeneracion}\n")


Problación generada:
[[4.461915335497425], [-4.997010481579528], [3.451441764343919], [-0.8972084887064327]]

De 4 individuos.

Las evaluaciones fueron:
[39.623746328981525, 24.97187783773981, 31.450618257490053, 2.819147309001826]

El individuo más apto fue [-0.8972084887064327]

Las probabilidades de selección fueron
[1.0374465644847263e-16, 2.3943706051045154e-10, 3.6771399414708667e-13, 0.9999999997601952]

Las probabilidades acumulativas fueron
[1.0374465644847263e-16, 2.39437164255108e-10, 2.398048782492551e-10, 1.0]

Los padres de la siguiente generación son:
[([-0.8972084887064327], [-0.8972084887064327]), ([-0.8972084887064327], [-0.8972084887064327])]

Tenemos 2 parejas de padres, pues queremos generar 4 nuevos individuos.

Se permitirán 4 clones

Los padres de la siguiente generación (como vectores binarios) son:
[([[0, 1, 1]], [[0, 1, 1]]), ([[0, 1, 1]], [[0, 1, 1]])]

Los hijos generados en la cruza (como vectores binarios) son:
[[[0, 1, 1]], [[0, 1, 1]], [[0, 1, 1]]]

Se 

**LA IMPLEMENTACIÓN**

Ya tienes todos los ingredientes... Vamos a cocinar:

1. Quisiera que el algoritmo estuviera 'en una sola celda'.
2. Podrías llamarlo `Algoritmo_Genetico` (esta cosa hace una ejecución)
3. Debe tener ciertos parámetros:
    * `funcion`: Función objetivo en la que vas a evaluar tus cosas.
    * `intervalo`: Intervalo de tu `funcion`
    * `dimension`: Dimension en la que está tu problema.
    * `kindividuos`: $2(kindividuos)$ va a ser la cantidad de individuos con la que trabajarás. (o sea, `kindividuos` es la mitad del tamaño de la población). 
    * `nBits`: La cantidad de bits que quieres para la codificación en arreglos binarios.
    * `iteraciones`: Número de iteraciones que quieres para tu algoritmo.
    * `nCortes`: Número de cortes para el operador de reproducción de cruza.
    * `probaMutar`: La probabilidad que tiene un hijo de mutar.

In [27]:
def Algoritmo_Genetico(funcion, intervalo, dimension,   # Cuando tienes una función que
                       kindividuos, nBits, nCortes,     # tiene muchos parámetros, la
                       probaMutar, iteraciones):        # puedes poner así para que sea
                                                        # más legible.
    
    contadorIteraciones = 0 # Contamos las iteraciones.

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo) 
    
    while contadorIteraciones <= iteraciones:

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres (ponemos `_` porque no nos interesa el número de 
        # clones).
        padres, _ = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruzar_N_Puntos(padresBinarios, nCortes, nBits)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)

        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Siguiente_Gen(hijosDecodificados, masApto)

        contadorIteraciones += 1 # Incrementamos en uno el contador de las iteraciones.

    return masApto # Devolvemos la mejor solución encontrada.


In [46]:
'''Veamos un ejemplo de implementación'''

funcion = Rastrigin
intervalo = (-5, 5)
dimension = 10
kindividuos = 70 # Tendremos una población del doble
nBits = 15
nCortes = 14
probaMutar = 0.45
iteraciones = 1000

masApto = Algoritmo_Genetico(funcion, intervalo, dimension,   
                             kindividuos, nBits, nCortes,     
                             probaMutar, iteraciones)

print(f"El individuo más apto fue:\n{masApto}\n")
print(f"Con una evaluacion de {funcion(masApto)}")



El individuo más apto fue:
[-0.99517822265625, 0.994873046875, -0.00030517578125, 0.0, -0.00030517578125, -1.99249267578125, -1.875, 1.9921875, 0.0, 0.0]

Con una evaluacion de 16.396529744928145
